# Elastic Net Regression

In [2]:
# extension that autoformats all code to PEP8
%load_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
import numpy as np
import pandas as pd
import plotly.express as px
import statsmodels.api as sm
from sklearn.linear_model import ElasticNet
from IPython.display import display
import plotly.express as px
import plotly.graph_objects as go
import time

C:\Users\Gianni\anaconda3\lib\site-packages\statsmodels\compat\pandas.py:61: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


<IPython.core.display.Javascript object>

The advantage of Elastic Net is that we get the best of both worlds: the dimnesion reduction by shrinkage without ignoring highly correlated variables in groups.Our goal in Elastic Net is to minimize the following loss function:
$$
1 / (2 * n_{samples}) * ||y - Xw||^2_2
+ alpha * l_1 ratio * ||w||_1
+ 0.5 * alpha * (1 - l_1 ratio) * ||w||^2_2
$$ 
or 
$$
a * ||w||_1 + 0.5 * b * ||w||_2^2
$$

## Data Loading and Cleaning

In [4]:
# Using updated dataframe from preprocessing notebook
df = pd.read_csv("../data/listings.csv")
# Random column of index
df = df.drop(["Unnamed: 0"], axis=1,)
display(df)

,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,neighbourhood_group_Bronx,neighbourhood_group_Brooklyn,neighbourhood_group_Manhattan,neighbourhood_group_Queens,...,neighbourhood_Williamsburg,neighbourhood_Willowbrook,neighbourhood_Windsor Terrace,neighbourhood_Woodhaven,neighbourhood_Woodlawn,neighbourhood_Woodside,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,days_since_review
0,149,1,9,0.21,6,365,0,1,0,0,...,0,0,0,0,0,0,0,1,0,1235
1,225,1,45,0.38,2,355,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1021
2,89,1,270,4.64,1,194,0,1,0,0,...,0,0,0,0,0,0,1,0,0,976
3,80,10,9,0.10,1,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1204
4,200,3,74,0.59,1,129,0,0,1,0,...,0,0,0,0,0,0,1,0,0,989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38832,129,1,1,1.00,1,147,0,0,1,0,...,0,0,0,0,0,0,0,1,0,974
38833,45,1,1,1.00,6,339,0,0,0,1,...,0,0,0,0,0,0,0,1,0,974
38834,235,1,1,1.00,1,87,0,0,0,0,...,0,0,0,0,0,0,0,1,0,974
38835,100,1,2,2.00,1,40,1,0,0,0,...,0,0,0,0,0,0,1,0,0,974


<IPython.core.display.Javascript object>

In [5]:
# Who needs a black box?
def standardizeData(data):
    # Takes pandas dataframe and standardizes it iteratively
    for column in data:
        if (data[column] != 0).sum() == 0:  # If the column is all zeros
            continue  # No need to standardize all zeros (thus dividing by zero)
        else:
            data[column] = (data[column] - np.mean(data[column])) / np.std(data[column])
    return data

<IPython.core.display.Javascript object>

In [6]:
# Create training data, holding out 10% of data for testing
from sklearn.model_selection import train_test_split  # Returns pandas dataframe

Xtrain, Xtest, Ytrain, Ytest = train_test_split(
    df.drop("price", axis=1), df["price"], test_size=0.2, random_state=0
)
# display(Xtrain["neighbourhood_Willowbrook"])
# # Standardize data for faster convergence in gradient descent
display(Xtest.isnull().values.any())
print((Xtest["neighbourhood_Willowbrook"] != 0).sum())
Xtrain, Xtest = standardizeData(Xtrain), standardizeData(Xtest)

# Standardize data for faster convergence in gradient descent
# from sklearn.preprocessing import StandardScaler

# sc = StandardScaler()
# Xtrain, Xtest = sc.fit_transform(Xtrain), sc.fit_transform(Xtest)
display(Ytest)
print(Xtrain.isnull().values.any())
display(Xtrain)
display(Xtest.isnull().values.any())
display(Xtest)

False

0


17693     77
9252      98
37779     60
32228    230
10427    310
        ... 
36045    160
959       88
18321    180
30295     59
33738     45
Name: price, Length: 7768, dtype: int64

False


,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,neighbourhood_group_Bronx,neighbourhood_group_Brooklyn,neighbourhood_group_Manhattan,neighbourhood_group_Queens,neighbourhood_group_Staten Island,...,neighbourhood_Williamsburg,neighbourhood_Willowbrook,neighbourhood_Windsor Terrace,neighbourhood_Woodhaven,neighbourhood_Woodlawn,neighbourhood_Woodside,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,days_since_review
10436,-0.269916,2.551986,1.223461,-0.158139,0.657345,-0.149250,-0.857333,1.154040,-0.365082,-0.090428,...,-0.295945,-0.005673,-0.056825,-0.043619,-0.017022,-0.064821,0.956990,-0.916594,-0.148119,-0.630421
11103,-0.269916,-0.584464,-0.799304,-0.158139,-0.885481,-0.149250,-0.857333,1.154040,-0.365082,-0.090428,...,-0.295945,-0.005673,-0.056825,-0.043619,-0.017022,-0.064821,0.956990,-0.916594,-0.148119,2.424477
35112,-0.159753,-0.439066,0.612479,0.154062,-0.499774,-0.149250,-0.857333,1.154040,-0.365082,-0.090428,...,-0.295945,-0.005673,-0.056825,-0.043619,-0.017022,-0.064821,-1.044943,1.090996,-0.148119,-0.543893
10110,0.446140,-0.584464,-0.799304,-0.158139,-0.885481,-0.149250,-0.857333,1.154040,-0.365082,-0.090428,...,-0.295945,-0.005673,-0.056825,-0.043619,-0.017022,-0.064821,-1.044943,1.090996,-0.148119,2.414863
37730,-0.269916,-0.501379,1.662418,-0.158139,0.402779,-0.149250,-0.857333,-0.866521,2.739112,-0.090428,...,-0.295945,-0.005673,-0.056825,-0.043619,-0.017022,-0.064821,-1.044943,1.090996,-0.148119,-0.652053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20757,-0.159753,-0.397523,-0.562030,-0.158139,0.850199,-0.149250,1.166408,-0.866521,-0.365082,-0.090428,...,-0.295945,-0.005673,-0.056825,-0.043619,-0.017022,-0.064821,0.956990,-0.916594,-0.148119,-0.334785
32103,-0.159753,-0.210583,0.725184,-0.158139,-0.044641,6.700162,-0.857333,-0.866521,-0.365082,-0.090428,...,-0.295945,-0.005673,-0.056825,-0.043619,-0.017022,-0.064821,-1.044943,1.090996,-0.148119,-0.587157
30403,-0.214834,-0.314439,0.416728,0.310162,1.212763,-0.149250,-0.857333,1.154040,-0.365082,-0.090428,...,-0.295945,-0.005673,-0.056825,-0.043619,-0.017022,-0.064821,0.956990,-0.916594,-0.148119,-0.620807
21243,-0.269916,-0.355981,-0.496779,-0.080089,-0.885481,-0.149250,1.166408,-0.866521,-0.365082,-0.090428,...,3.379004,-0.005673,-0.056825,-0.043619,-0.017022,-0.064821,0.956990,-0.916594,-0.148119,0.436750


False

,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,neighbourhood_group_Bronx,neighbourhood_group_Brooklyn,neighbourhood_group_Manhattan,neighbourhood_group_Queens,neighbourhood_group_Staten Island,...,neighbourhood_Williamsburg,neighbourhood_Willowbrook,neighbourhood_Windsor Terrace,neighbourhood_Woodhaven,neighbourhood_Woodlawn,neighbourhood_Woodside,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,days_since_review
17693,-0.197345,-0.536615,-0.757874,-0.160144,-0.892507,-0.162146,1.168815,-0.860316,-0.366532,-0.089698,...,-0.30497,0,-0.060146,-0.040943,-0.016048,-0.071944,0.943411,-0.900636,-0.15358,-0.103330
9252,-0.341340,3.536798,1.792199,-0.160144,1.376214,-0.162146,1.168815,-0.860316,-0.366532,-0.089698,...,-0.30497,0,-0.060146,-0.040943,-0.016048,-0.071944,0.943411,-0.900636,-0.15358,-0.625232
37779,-0.341340,-0.309165,5.324924,-0.021386,-0.203373,-0.162146,-0.855567,-0.860316,2.728274,-0.089698,...,-0.30497,0,-0.060146,-0.040943,-0.016048,-0.071944,-1.059983,1.110327,-0.15358,-0.657387
32228,-0.269343,-0.350520,0.230807,-0.160144,-0.892507,-0.162146,-0.855567,1.162364,-0.366532,-0.089698,...,-0.30497,0,-0.060146,-0.040943,-0.016048,-0.071944,0.943411,-0.900636,-0.15358,-0.365518
10427,-0.197345,1.799911,0.893946,-0.160144,-0.451152,-0.162146,-0.855567,1.162364,-0.366532,-0.089698,...,-0.30497,0,-0.060146,-0.040943,-0.016048,-0.071944,0.943411,-0.900636,-0.15358,-0.620285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36045,-0.269343,-0.557292,-0.161049,-0.160144,-0.420179,-0.162146,-0.855567,1.162364,-0.366532,-0.089698,...,-0.30497,0,-0.060146,-0.040943,-0.016048,-0.071944,-1.059983,1.110327,-0.15358,-0.640073
959,-0.197345,4.405241,0.815575,-0.160144,0.787740,-0.162146,1.168815,-0.860316,-0.366532,-0.089698,...,-0.30497,0,16.626141,-0.040943,-0.016048,-0.071944,0.943411,-0.900636,-0.15358,-0.625232
18321,0.594630,-0.598647,-0.781988,-0.160144,-0.892507,-0.162146,1.168815,-0.860316,-0.366532,-0.089698,...,-0.30497,0,-0.060146,-0.040943,-0.016048,-0.071944,0.943411,-0.900636,-0.15358,0.067339
30295,-0.341340,-0.019684,0.960260,-0.160144,1.020033,-0.162146,-0.855567,1.162364,-0.366532,-0.089698,...,-0.30497,0,-0.060146,-0.040943,-0.016048,-0.071944,-1.059983,1.110327,-0.15358,-0.602971


<IPython.core.display.Javascript object>

## Performing Elastic Net Regression

In [7]:
# doing Elastic Net by hand via a class
# https://math.stackexchange.com/questions/1111504/differentiation-with-respect-to-a-matrix-residual-sum-of-squares
class ElasticRegression:
    def __init__(self, learning_rate, iterations, l1_penalty, l2_penalty):

        self.learning_rate = learning_rate
        self.iterations = iterations
        self.l1_penalty = l1_penalty
        self.l2_penalty = l2_penalty

    # Function for model training
    def fit(self, X, Y):

        # no_of_training_examples, no_of_features
        self.m, self.n = X.shape

        # weight initialization
        self.W = np.zeros(self.n)
        self.b = 0
        self.X = X
        self.Y = Y

        # gradient descent learning
        for i in range(self.iterations):
            self.update_weights()

        return self

    # Helper function to update weights in gradient descent
    def update_weights(self):
        Y_pred = self.predict(self.X)

        # calculate gradients
        dW = np.zeros(self.n)

        for j in range(self.n):
            if self.W[j] > 0:
                dW[j] = (
                    -(2 * np.transpose(self.X[:, j]) @ (self.Y - Y_pred))
                    + self.l1_penalty
                    + 2 * self.l2_penalty * self.W[j]
                ) / self.m
            else:
                dW[j] = (
                    -(2 * np.transpose(self.X[:, j]) @ (self.Y - Y_pred))
                    - self.l1_penalty
                    + 2 * self.l2_penalty * self.W[j]
                ) / self.m
        db = -2 * np.sum(self.Y - Y_pred) / self.m

        # update weights
        self.W = self.W - self.learning_rate * dW

        self.b = self.b - self.learning_rate * db

        return self

    # Hypothetical function  h( x )
    def predict(self, X):
        return X @ self.W + self.b

<IPython.core.display.Javascript object>

### Testing with the 'Iris' Data Set

In [8]:
# Load in iris from seaborn library
iris = pd.read_csv(
    "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv"
)
iris = iris.drop("species", axis=1)
display(iris)

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


<IPython.core.display.Javascript object>

In [9]:
Xtrain_iris, Xtest_iris, Ytrain_iris, Ytest_iris = train_test_split(
    iris.drop("sepal_length", axis=1),
    iris["sepal_length"],
    test_size=0.2,
    random_state=42,
)

# Standardize our X matrices
Xtrain_iris, Xtest_iris = standardizeData(Xtrain_iris), standardizeData(Xtest_iris)

# Do the model
model = ElasticRegression(
    iterations=100, learning_rate=0.01, l1_penalty=0.99, l2_penalty=0.01
)
model.fit(Xtrain_iris.to_numpy(), Ytrain_iris.to_numpy())

Y_pred_iris = model.predict(Xtest_iris)
display(pd.DataFrame({"Predictions": Y_pred_iris, "Actual": Ytest_iris}).head())

,Predictions,Actual
73,5.112736,6.1
18,4.446238,5.7
118,5.964557,7.7
78,5.219452,6.0
76,5.212697,6.8


<IPython.core.display.Javascript object>

### On to Our AirBnB Data

In [10]:
enm = ElasticRegression(
    iterations=1000, learning_rate=0.01, l1_penalty=0.5, l2_penalty=0.5
)

# display(Xtrain)
enm.fit(Xtrain.to_numpy(), Ytrain.to_numpy())

Ypred = enm.predict(Xtest)

display(pd.DataFrame({"Predictions": np.round(Ypred, 2), "Actual": Ytest}))

# print("Trained W: ", np.round(enm.W, 2))
# print("Trained b: ", np.round(enm.b, 2))

,Predictions,Actual
17693,160.39,77
9252,156.09,98
37779,63.28,60
32228,174.79,230
10427,284.09,310
...,...,...
36045,123.02,160
959,139.43,88
18321,181.37,180
30295,111.26,59


<IPython.core.display.Javascript object>

## Hyperparameter Selection with KFold Cross Validation

In [11]:
dataX = standardizeData(df.drop("price", axis=1))
dataY = df["price"]

<IPython.core.display.Javascript object>

In [12]:
import multiprocessing as mp

pool = mp.Pool(mp.cpu_count())

<IPython.core.display.Javascript object>

In [19]:
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


def lambda_tune(X_train, y_train):

    all_mse = np.array(
        []
    )  # 200 total entries (lambda), each entry has 10 entries (10-fold)
    all_mae = np.array([])
    # lambda1 = np.array([1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.0, 0.5, 1.0, 10.0, 100.0])
    lambda1 = np.arange(0.01, 1.01, 0.05)
    # lambda1 = np.arange(0, 0.03, 0.01)
    # lambda2 = np.array([1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.0, 0.5, 1.0, 10.0, 100.0])
    lambda2 = np.flip(np.arange(0.01, 1.01, 0.05))  # Range of lambda values
    # lambda2 = np.arange(0, 0.03, 0.01)

    # TODO: Finish this cross validation wiht the lambdas in a nested for loop (try to match sklearn so we can still graph)
    # kf = KFold(n_splits=5, random_state=42, shuffle=True)

    for l1 in lambda1:
        for l2 in lambda2:  # Iterate through all lambda (200)
            #             MSE = np.array([])
            #             MAE = np.array([])
            #             for train_index, test_index in kf.split(
            #                 X_train
            #             ):  # Iterate through each of 10 folds for each lamda
            #                 temp_X_train = X_train[train_index]
            #                 temp_y_train = y_train[train_index]
            #                 temp_X_test = X_train[test_index]
            #                 temp_y_test = y_train[test_index]
            #                         for i in range(5):  # 5 (k) fold cross validation

            # Split the data
            Xtrain, Xtest, Ytrain, Ytest = train_test_split(
                X_train, y_train, test_size=0.2,
            )
            # Build model with new parameters
            model = ElasticRegression(
                iterations=100, learning_rate=0.01, l1_penalty=l1, l2_penalty=l2
            )
            # fit model
            model.fit(Xtrain, Ytrain)
            # Prediction
            Ypred = model.predict(
                Xtest
            )  # Predicted response values based on beta coefficients

            #             MSE = np.append(MSE, mean_squared_error(y_true=Ytest, y_pred=Ypred))
            #             MAE = np.append(MAE, mean_absolute_error(y_true=Ytest, y_pred=Ypred))

            #             all_mse = np.append(all_mse, MSE)
            #             all_mae = np.append(all_mae, MAE)
            all_mse = np.append(all_mse, mean_squared_error(y_true=Ytest, y_pred=Ypred))
            all_mae = np.append(
                all_mae, mean_absolute_error(y_true=Ytest, y_pred=Ypred)
            )

    return all_mae, all_mse


start = time.time()
all_mae, all_mse = lambda_tune(dataX.to_numpy(), dataY.to_numpy())

print("--- %s seconds ---" % (time.time() - start))

--- 900.6981947422028 seconds ---


<IPython.core.display.Javascript object>

In [21]:
# --- 900.6981947422028 seconds ---
# Next we take the mae and mse and nest it for for every 5 entries so we isolate each kfold
# MSE = np.array([all_mse[i : i + 5] for i in range(0, len(all_mse), 5)])
# MAE = np.array([all_mae[i : i + 5] for i in range(0, len(all_mae), 5)])

# Make dataframes
kFoldsMSE = pd.DataFrame(np.array(all_mse))
kFoldsMAE = pd.DataFrame(np.array(all_mae))

# Get the minimum kfold
minkFoldsMSE = pd.DataFrame(kFoldsMSE.min(axis=1), columns=["minMSE"])
minkFoldsMAE = pd.DataFrame(kFoldsMAE.min(axis=1), columns=["minMAE"])


l1col = np.concatenate([np.repeat(x, 20) for x in np.arange(0.01, 1.01, 0.05)])
l2col = np.concatenate(
    [np.arange(0.01, 1.01, 0.05) for x in np.arange(0.01, 1.01, 0.05)]
)

graphdf = pd.DataFrame({"l1": l1col, "l2": l2col})
graphdf = graphdf.join(minkFoldsMSE).join(minkFoldsMAE)

,0
0,30323.999079
1,32468.414758
2,56508.525912
3,46246.635414
4,15436.790690
...,...
395,36755.899378
396,36911.556795
397,19189.532011
398,42081.941668


,minMSE
0,30323.999079
1,32468.414758
2,56508.525912
3,46246.635414
4,15436.790690
...,...
395,36755.899378
396,36911.556795
397,19189.532011
398,42081.941668


,l1,l2,minMSE,minMAE
0,0.01,0.01,30323.999079,57.192124
1,0.01,0.06,32468.414758,55.334149
2,0.01,0.11,56508.525912,60.132174
3,0.01,0.16,46246.635414,57.073007
4,0.01,0.21,15436.790690,53.327096
...,...,...,...,...
395,0.96,0.76,36755.899378,55.487788
396,0.96,0.81,36911.556795,58.184912
397,0.96,0.86,19189.532011,53.735951
398,0.96,0.91,42081.941668,57.015009


<IPython.core.display.Javascript object>

In [54]:
# Plotting 3 dimensional plot
figMSE = go.Figure(
    data=px.scatter_3d(
        graphdf,
        x="l1",
        y="l2",
        z="minMSE",
        opacity=1,
        labels={"l1": "LASSO parameter", "l2": "Ridge parameter", "minMSE": "MSE",},
        color_discrete_sequence=px.colors.qualitative.Safe,
        title="MSE with Combined LASSO and Ridge Penalty Values",
    )
)


minMSEpoint = graphdf.drop("minMAE", axis=1).min()
figMSE.add_trace(
    go.Scatter3d(
        x=[minMSEpoint["l1"]],
        y=[minMSEpoint["l2"]],
        z=[minMSEpoint["minMSE"]],
        mode="markers",
        marker_symbol="circle",
        marker_size=10,
    )
)
figMSE.show()


figMAE = go.Figure(
    data=px.scatter_3d(
        graphdf,
        x="l1",
        y="l2",
        z="minMAE",
        opacity=1,
        labels={"l1": "LASSO parameter", "l2": "Ridge parameter", "minMAE": "MAE",},
        color_discrete_sequence=px.colors.qualitative.Pastel1,
        title="MAE in Combined LASSO and Ridge Penalty Values",
    )
)


minMAEpoint = graphdf.drop("minMSE", axis=1).min()

figMAE.add_trace(
    go.Scatter3d(
        x=[minMAEpoint["l1"]],
        y=[minMAEpoint["l2"]],
        z=[minMAEpoint["minMAE"]],
        mode="markers",
        marker_symbol="circle",
        marker_size=10,
    )
)
figMAE.show()


figMSE.write_html("../data/3dMSE.html")
figMAE.write_html("../data/3dMAE.html")

<IPython.core.display.Javascript object>

In [ ]:
# ### WARNING: BE CAREFUL WHEN RUNNING THIS, IT WILL USE YOUR ENTIRE CPU!!!

# # # Performing Elastic Net with sklearn and CV
# from sklearn.linear_model import ElasticNetCV
# from sklearn.model_selection import RepeatedKFold
# import warnings

# # Muting the 1000 warnings this cell will output
# warnings.filterwarnings("ignore")

# # Measure run time
# start = time.time()

# # Method of model eval
# cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)

# # Defining lists for model eval
# ratios = np.arange(0, 1, 0.01)
# alphas = np.arange(
#     0, 1, 0.01
# )  # [0.0, 1e-5, 1e-4, 1e-3, 1e-2, 0.1, 0.5, 0.75, 1.0, 10.0, 100.0]


# enm = ElasticNetCV(
#     l1_ratio=ratios, alphas=alphas, fit_intercept=False, cv=cv, n_jobs=-1
# )  # n_jobs = -1 will use all processors on CPU
# enm.fit(Xtrain, Ytrain)


# print("--- %s seconds ---" % (time.time() - start))

In [ ]:
# Lets take the coefficients of training and the names of our columns and put them into a dataframe
variables = Xtrain.columns.tolist()
model_coef = pd.DataFrame({"coef": enm.coef_, "names": variables})

print("alpha: %f" % enm.alpha_)
print("l1_ratio_: %f" % enm.l1_ratio_)

# show all the coef that are non zero in descending order
display(model_coef[model_coef["coef"] != 0].sort_values(by="coef", ascending=False))


# print(enm.mse_path_[0][0])
enm.alphas_

In [ ]:
# These next three cells will be dedicated to plotting values used in CV
MSE = []
#l1col = np.concatenate([np.repeat(x, 100) for x in np.arange(0, 1, 0.01)])
l1
alphacol = np.flip(
    np.concatenate([np.arange(0, 1, 0.01) for x in np.arange(0, 1, 0.01)])
)
for i in range(len(enm.mse_path_)):
    for j in range(len(enm.mse_path_[i])):
        MSE.append(enm.mse_path_[i][j])

In [ ]:
kFolds = pd.DataFrame(np.array(MSE))
minkFolds = pd.DataFrame(kFolds.min(axis=1), columns=["min"])
graphdf = pd.DataFrame({"l1": l1col, "alpha": alphacol})
graphdf = graphdf.join(minkFolds)
# display(graphdf)

In [ ]:
# Plotting 3 dimensional plot
fig = go.Figure(
    data=px.scatter_3d(
        graphdf,
        x="l1",
        y="alpha",
        z="min",
        opacity=0.05,
        labels={
            "l1": "LASSO parameter",
            "alpha": "Ridge parameter",
            "min": "Minimumn MSE",
        },
        color_discrete_sequence=px.colors.qualitative.Safe,
        title="Minimum MSE in 3 Trial 10-Fold CV with Combined LASSO and Ridge Penalty Values",
    )
)
fig.show()

fig.write_html("../data/3dMSE.html")

In [ ]:
# Now lets predict with our testing data
predictions = enm.predict(Xtest)

# Compared to the actual values
pred_v_act = pd.DataFrame({"predictions": enm.predict(Xtest), "actual": Ytest})
display(pred_v_act)

from sklearn.metrics import *

# Metrics
score = r2_score(Ytest, predictions)
MAE = mean_absolute_error(Ytest, predictions)
MSE = mean_squared_error(Ytest, predictions)

# Printing of Metrics
print("The r-squared value is ", score)
print(
    "The adjusted r-squared value is ",
    1
    - (
        ((1 - score) * (len(pred_v_act.index) - 1))
        / (len(pred_v_act) - len(pred_v_act.columns) - 1)
    ),
)
print("The Mean Absolute Error is ", MAE)
print("The root Mean Squared Error is ", np.sqrt(MSE))